In [20]:
# IMPORTS
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
# CONFIGURATION
DATA_PATH    = 'data/COVID clinical trials.csv'  
OUTPUT_DIR   = 'outputs/figures'                 
CLEANED_PATH = 'outputs/cleaned_trials.csv'     
RANDOM_SEED  = 123                               

In [22]:
os.makedirs(OUTPUT_DIR, exist_ok=True)
np.random.seed(RANDOM_SEED)

LOAD & INITIAL INSPECTION

In [23]:
df = pd.read_csv(DATA_PATH)
print("1) Data loaded:")
print("  • rows:", df.shape[0], "columns:", df.shape[1])
print(df.info(), "\n")

1) Data loaded:
  • rows: 5783 columns: 27
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5783 entries, 0 to 5782
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Rank                     5783 non-null   int64  
 1   NCT Number               5783 non-null   object 
 2   Title                    5783 non-null   object 
 3   Acronym                  2480 non-null   object 
 4   Status                   5783 non-null   object 
 5   Study Results            5783 non-null   object 
 6   Conditions               5783 non-null   object 
 7   Interventions            4897 non-null   object 
 8   Outcome Measures         5748 non-null   object 
 9   Sponsor/Collaborators    5783 non-null   object 
 10  Gender                   5773 non-null   object 
 11  Age                      5783 non-null   object 
 12  Phases                   3322 non-null   object 
 13  Enrollment               5749 non-n

In [24]:
# Show a glimpse
print("First 5 rows:")
print(df.head(), "\n")

First 5 rows:
   Rank   NCT Number                                              Title  \
0     1  NCT04785898  Diagnostic Performance of the ID Now™ COVID-19...   
1     2  NCT04595136  Study to Evaluate the Efficacy of COVID19-0001...   
2     3  NCT04395482  Lung CT Scan Analysis of SARS-CoV2 Induced Lun...   
3     4  NCT04416061  The Role of a Private Hospital in Hong Kong Am...   
4     5  NCT04395924         Maternal-foetal Transmission of SARS-Cov-2   

        Acronym                  Status         Study Results  \
0   COVID-IDNow  Active, not recruiting  No Results Available   
1      COVID-19      Not yet recruiting  No Results Available   
2   TAC-COVID19              Recruiting  No Results Available   
3      COVID-19  Active, not recruiting  No Results Available   
4  TMF-COVID-19              Recruiting  No Results Available   

                                          Conditions  \
0                                            Covid19   
1                               

In [25]:
# DROP OVERLY-SPARSE COLUMNS
# Columns with >80% missing values are dropped
missing_pct = df.isnull().mean()
to_drop = missing_pct[missing_pct > 0.80].index.tolist()
print("2) Dropping columns with >80% missing:", to_drop)
df.drop(columns=to_drop, inplace=True)

2) Dropping columns with >80% missing: ['Results First Posted', 'Study Documents']


HANDLE MODERATELY MISSING CATEGORICALS

In [26]:
# Fill remaining NA in these string fields with 'Unknown'
for col in ['Acronym', 'Phases', 'Interventions']:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')
        print(f"  • Imputed '{col}' missing → 'Unknown'")

  • Imputed 'Acronym' missing → 'Unknown'
  • Imputed 'Phases' missing → 'Unknown'
  • Imputed 'Interventions' missing → 'Unknown'


DATE PARSING & FEATURE EXTRACTION

In [27]:
# Convert date strings to datetime, then extract year/month
for date_col in ['Start Date', 'Primary Completion Date', 'Completion Date']:
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        df[f'{date_col} Year'] = df[date_col].dt.year
        df[f'{date_col} Month'] = df[date_col].dt.month
        print(f"4) Parsed and extracted year/month from '{date_col}'")

4) Parsed and extracted year/month from 'Start Date'
4) Parsed and extracted year/month from 'Primary Completion Date'
4) Parsed and extracted year/month from 'Completion Date'


NUMERIC COLUMN CLEANUP

In [28]:
if 'Enrollment' in df.columns:
    df['Enrollment'] = pd.to_numeric(df['Enrollment'], errors='coerce')
    
    df.loc[df['Enrollment'] < 0, 'Enrollment'] = np.nan
    enroll_median = int(df['Enrollment'].median())
    df['Enrollment'] = df['Enrollment'].fillna(enroll_median)
    print(f"5) Cleaned 'Enrollment', imputed missing with median = {enroll_median}")

5) Cleaned 'Enrollment', imputed missing with median = 170


In [29]:
# UNIVARIATE ANALYSIS (VALUE COUNTS & BARCHARTS)
# Helper to save and close figures
def save_plot(name):
    path = os.path.join(OUTPUT_DIR, name + '.png')
    plt.tight_layout()
    plt.savefig(path)
    plt.close()
    print(f"   • Saved plot: {path}")

In [30]:
# Study Status distribution
plt.figure(figsize=(6,4))
df['Status'].value_counts().plot(kind='bar')
plt.title('Study Status Distribution')
save_plot('univariate_status')

   • Saved plot: outputs/figures\univariate_status.png


In [31]:
# Phases distribution
if 'Phases' in df.columns:
    plt.figure(figsize=(6,4))
    df['Phases'].value_counts().plot(kind='bar', color='skyblue')
    plt.title('Clinical Trial Phases')
    save_plot('univariate_phases')

   • Saved plot: outputs/figures\univariate_phases.png


In [32]:
#  Trial Design types
if 'Study Designs' in df.columns:
    plt.figure(figsize=(6,4))
    df['Study Designs'].value_counts().nlargest(10).plot(kind='barh')
    plt.title('Top 10 Study Designs')
    save_plot('univariate_designs')

C:\Users\DEVESH RAI\AppData\Local\Temp\ipykernel_21000\3666006153.py:5: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


   • Saved plot: outputs/figures\univariate_designs.png


In [33]:
# BIVARIATE ANALYSIS
# Status vs Phase
if {'Status','Phases'}.issubset(df.columns):
    cross = pd.crosstab(df['Phases'], df['Status'], normalize='index')
    cross.plot(kind='bar', stacked=True, figsize=(7,5))
    plt.title('Study Phases vs Status (proportion)')
    save_plot('bivariate_phase_status')

   • Saved plot: outputs/figures\bivariate_phase_status.png


In [34]:
# TIME-SERIES TREND
# Monthly count of trials started
if 'Start Date Year' in df.columns:
    monthly = df.groupby(['Start Date Year','Start Date Month']).size().reset_index(name='Count')
    monthly = monthly.rename(columns={'Start Date Year': 'year', 'Start Date Month': 'month'})
    monthly['day'] = 1
    monthly['Date'] = pd.to_datetime(monthly[['year', 'month', 'day']])
    plt.figure(figsize=(8,4))
    plt.plot(monthly['Date'], monthly['Count'], marker='o')
    plt.title('Monthly Number of COVID-19 Trial Starts')
    plt.xlabel('Date')
    plt.ylabel('Number of Trials')
    save_plot('timeseries_monthly_trials')

   • Saved plot: outputs/figures\timeseries_monthly_trials.png


In [35]:
# FEATURE ENGINEERING (COUNTRY EXTRACTION)
# If 'Locations' column exists, extract first country seen
if 'Locations' in df.columns:
    def extract_country(loc):
        return loc.split(';')[0].split(',')[-1].strip()
    df['Country'] = df['Locations'].fillna('').apply(extract_country)
    top_countries = df['Country'].value_counts().nlargest(10)
    plt.figure(figsize=(6,4))
    top_countries.plot(kind='bar')
    plt.title('Top 10 Countries by Trial Count')
    save_plot('feature_country_counts')

   • Saved plot: outputs/figures\feature_country_counts.png


In [36]:
#  TOP 10 INTERVENTIONS
if 'Interventions' in df.columns:
    top_interventions = df['Interventions'].str.split('|').explode().value_counts().nlargest(10)
    plt.figure(figsize=(8,4))
    top_interventions.plot(kind='bar', color='teal')
    plt.title('Top 10 Interventions in COVID-19 Trials')
    plt.ylabel('Number of Trials')
    save_plot('top_interventions')

   • Saved plot: outputs/figures\top_interventions.png


In [37]:
# SAVE CLEANED DATA
df.to_csv(CLEANED_PATH, index=False)
print(f"10) Cleaned dataset saved to {CLEANED_PATH}")

10) Cleaned dataset saved to outputs/cleaned_trials.csv


In [38]:
print("11) EDA complete — all figures are in", OUTPUT_DIR)
print("    You can now proceed to deeper analysis or model building.")

11) EDA complete — all figures are in outputs/figures
    You can now proceed to deeper analysis or model building.
